<a href="https://colab.research.google.com/github/jmohsbeck1/jpmc_mle/blob/final_project/DataDazzlers_DineOracle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Library Imports

In [ ]:
import os
import warnings
import itertools
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 
warnings.filterwarnings('ignore')

In [ ]:
#Dataframe and numerical library
import pandas as pd 
pd.set_option('display.max_columns', None)
import numpy as np

#Visualization
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
%matplotlib inline

# SQLite
import sqlite3

#Machine Learming Model
#Metrics
from sklearn.metrics import mutual_info_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score

#Model Selection
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

#Preprocessing
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

#Linear Model
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Ridge

#Ensemble
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

#Others
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeClassifier

#Hyper parameter
from sklearn import neighbors, datasets, model_selection

# Data Import

In [ ]:
#STEP 1: Import large dataset using opendatasets
if not os.path.exists('yelp-business-dataset'):
  print("Loading Keys")
  kaggle_key = 'ravikiranbutti'
  kaggle_value = '117268fa41345f39e5baeda66733a0c7'
  os.environ['KAGGLE_USERNAME'] = kaggle_key
  os.environ['KAGGLE_KEY'] = kaggle_value
  !mkdir -p /root/.kaggle
  with open('/root/.kaggle/kaggle.json', 'w') as kaggle_file:
    kaggle_file.write('{"username":"' + kaggle_key + '","key":"' + kaggle_value + '"}')

  print("Loading Data")
  !pip install kaggle
  !kaggle datasets download ravikiranbutti/yelp-business-dataset
  !unzip yelp-business-dataset.zip -d 'yelp-business-dataset'
  !rm yelp-business-dataset.zip

In [ ]:
#STEP 2: Load Business using chunk processing
business_chunks = pd.read_json("yelp-business-dataset/yelp_academic_dataset_business.json", lines=True, chunksize=1000000)
business = pd.concat(business_chunks) 

In [ ]:
#Print information
business.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150346 entries, 0 to 150345
Data columns (total 14 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   business_id   150346 non-null  object 
 1   name          150346 non-null  object 
 2   address       150346 non-null  object 
 3   city          150346 non-null  object 
 4   state         150346 non-null  object 
 5   postal_code   150346 non-null  object 
 6   latitude      150346 non-null  float64
 7   longitude     150346 non-null  float64
 8   stars         150346 non-null  float64
 9   review_count  150346 non-null  int64  
 10  is_open       150346 non-null  int64  
 11  attributes    136602 non-null  object 
 12  categories    150243 non-null  object 
 13  hours         127123 non-null  object 
dtypes: float64(3), int64(2), object(9)
memory usage: 16.1+ MB


# Data Cleaning


In [ ]:
#Select only the restaurant
business = business[business['categories'].str.contains('Restaurants', na=False)]
business.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 52268 entries, 3 to 150340
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   business_id   52268 non-null  object 
 1   name          52268 non-null  object 
 2   address       52268 non-null  object 
 3   city          52268 non-null  object 
 4   state         52268 non-null  object 
 5   postal_code   52268 non-null  object 
 6   latitude      52268 non-null  float64
 7   longitude     52268 non-null  float64
 8   stars         52268 non-null  float64
 9   review_count  52268 non-null  int64  
 10  is_open       52268 non-null  int64  
 11  attributes    51703 non-null  object 
 12  categories    52268 non-null  object 
 13  hours         44990 non-null  object 
dtypes: float64(3), int64(2), object(9)
memory usage: 6.0+ MB


In [ ]:
# Drop unimportant columns
business = business.drop(['business_id','name','address','latitude','longitude','hours'], axis=1)
business.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 52268 entries, 3 to 150340
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   city          52268 non-null  object 
 1   state         52268 non-null  object 
 2   postal_code   52268 non-null  object 
 3   stars         52268 non-null  float64
 4   review_count  52268 non-null  int64  
 5   is_open       52268 non-null  int64  
 6   attributes    51703 non-null  object 
 7   categories    52268 non-null  object 
dtypes: float64(1), int64(2), object(5)
memory usage: 3.6+ MB


In [ ]:
business.head()

,city,state,postal_code,stars,review_count,is_open,attributes,categories
3,Philadelphia,PA,19107,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B..."
5,Ashland City,TN,37015,2.0,6,1,"{'BusinessParking': 'None', 'BusinessAcceptsCr...","Burgers, Fast Food, Sandwiches, Food, Ice Crea..."
8,Affton,MO,63123,3.0,19,0,"{'Caters': 'True', 'Alcohol': 'u'full_bar'', '...","Pubs, Restaurants, Italian, Bars, American (Tr..."
9,Nashville,TN,37207,1.5,10,1,"{'RestaurantsAttire': ''casual'', 'Restaurants...","Ice Cream & Frozen Yogurt, Fast Food, Burgers,..."
11,Tampa Bay,FL,33602,4.0,10,1,"{'Alcohol': ''none'', 'OutdoorSeating': 'None'...","Vietnamese, Food, Restaurants, Food Trucks"


# Data Preprocessing - Attributes Column



In [ ]:
from pandas.io.json import json_normalize

# Use json_normalize() function to normalize the JSON column
normalized_df = json_normalize(business['attributes'])

# Merge the new DataFrame with the original DataFrame and fill missing values with 0
business_normalized = pd.concat([business, normalized_df], axis=1)
business_normalized.fillna(0, inplace=True)

# Set value to 1 if present and 0 if absent
business_normalized[normalized_df.columns] = business_normalized[normalized_df.columns].applymap(lambda x: 1 if x != 0 else 0)

#drop attributes column
business_normalized.drop(columns=['attributes'], inplace=True)

business_normalized.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 86405 entries, 3 to 52267
Data columns (total 46 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   city                        86405 non-null  object 
 1   state                       86405 non-null  object 
 2   postal_code                 86405 non-null  object 
 3   stars                       86405 non-null  float64
 4   review_count                86405 non-null  float64
 5   is_open                     86405 non-null  float64
 6   categories                  86405 non-null  object 
 7   RestaurantsDelivery         86405 non-null  int64  
 8   OutdoorSeating              86405 non-null  int64  
 9   BusinessAcceptsCreditCards  86405 non-null  int64  
 10  BusinessParking             86405 non-null  int64  
 11  BikeParking                 86405 non-null  int64  
 12  RestaurantsPriceRange2      86405 non-null  int64  
 13  RestaurantsTakeOut          864

In [ ]:
business_normalized.head()

,city,state,postal_code,stars,review_count,is_open,categories,RestaurantsDelivery,OutdoorSeating,BusinessAcceptsCreditCards,...,BestNights,BYOB,Corkage,BYOBCorkage,RestaurantsCounterService,Open24Hours,AgesAllowed,DietaryRestrictions,HairSpecializesIn,AcceptsInsurance
3,Philadelphia,PA,19107,4.0,80.0,1.0,"Restaurants, Food, Bubble Tea, Coffee & Tea, B...",1,1,1,...,0,0,0,0,0,0,0,0,0,0
5,Ashland City,TN,37015,2.0,6.0,1.0,"Burgers, Fast Food, Sandwiches, Food, Ice Crea...",1,1,1,...,0,0,0,0,0,0,0,0,0,0
8,Affton,MO,63123,3.0,19.0,0.0,"Pubs, Restaurants, Italian, Bars, American (Tr...",1,1,1,...,0,0,0,0,0,0,0,0,0,0
9,Nashville,TN,37207,1.5,10.0,1.0,"Ice Cream & Frozen Yogurt, Fast Food, Burgers,...",1,1,0,...,0,0,0,0,0,0,0,0,0,0
11,Tampa Bay,FL,33602,4.0,10.0,1.0,"Vietnamese, Food, Restaurants, Food Trucks",1,1,1,...,0,0,0,0,0,0,0,0,0,0


# Feature Selection - Attributes Column

In [ ]:
from sklearn.feature_selection import SelectKBest, f_classif

#NOTE: Add to sample it because of memory restrictions
business_feature_selection = business_normalized.sample(n=25000, random_state=42)

# Separate the features and the target variable
X = business_feature_selection.drop(['is_open','categories'], axis=1)
y = business_feature_selection['is_open']

# Assume df is the original dataframe with non-numerical columns
# Get all non-numericafal columns and perform one-hot encoding
non_numeric_cols = list(business_feature_selection.select_dtypes(exclude=['number', 'bool']).columns)
one_hot_encoded = pd.get_dummies(business_feature_selection[non_numeric_cols])

# Combine one-hot encoded columns with numerical columns
numerical_cols = list(business_feature_selection.select_dtypes(include=['number', 'bool']).columns)
X = pd.concat([business_feature_selection[numerical_cols], one_hot_encoded], axis=1)

# Select the top 10 features using f_classif score
selector = SelectKBest(score_func=f_classif, k=20)
X_new = selector.fit_transform(X, y)

# Get the selected feature names
selected_features = X.columns[selector.get_support()]

# Print the selected feature names
print('Selected features:', list(selected_features))

Selected features: ['stars', 'is_open', 'RestaurantsDelivery', 'OutdoorSeating', 'BusinessAcceptsCreditCards', 'BusinessParking', 'RestaurantsPriceRange2', 'RestaurantsTakeOut', 'WiFi', 'Alcohol', 'RestaurantsAttire', 'RestaurantsReservations', 'Ambience', 'GoodForKids', 'RestaurantsGoodForGroups', 'HasTV', 'city_0', 'state_0', 'postal_code_0', 'categories_0']


In [ ]:
#Data cleaning, remove columns already in the business set
selected_features = {item for item in selected_features if not item.startswith('categories')}
selected_features = {item for item in selected_features if not item.startswith('city_')}
selected_features = {item for item in selected_features if not item.startswith('state_')}
selected_features = {item for item in selected_features if not item.startswith('postal_code_')}
index = pd.Index(selected_features)

# select only columns in the Index object
business_normalized = pd.concat([business_normalized['categories'], business_normalized.loc[:, index]], axis=1)
business_normalized.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 86405 entries, 3 to 52267
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   categories                  86405 non-null  object 
 1   BusinessParking             86405 non-null  int64  
 2   is_open                     86405 non-null  float64
 3   OutdoorSeating              86405 non-null  int64  
 4   RestaurantsPriceRange2      86405 non-null  int64  
 5   RestaurantsTakeOut          86405 non-null  int64  
 6   WiFi                        86405 non-null  int64  
 7   BusinessAcceptsCreditCards  86405 non-null  int64  
 8   RestaurantsAttire           86405 non-null  int64  
 9   HasTV                       86405 non-null  int64  
 10  Ambience                    86405 non-null  int64  
 11  GoodForKids                 86405 non-null  int64  
 12  RestaurantsGoodForGroups    86405 non-null  int64  
 13  RestaurantsReservations     864

In [ ]:
business_normalized.head()

,categories,BusinessParking,is_open,OutdoorSeating,RestaurantsPriceRange2,RestaurantsTakeOut,WiFi,BusinessAcceptsCreditCards,RestaurantsAttire,HasTV,Ambience,GoodForKids,RestaurantsGoodForGroups,RestaurantsReservations,RestaurantsDelivery,stars,Alcohol
3,"Restaurants, Food, Bubble Tea, Coffee & Tea, B...",1,1.0,1,1,1,1,1,1,1,0,1,1,1,1,4.0,1
5,"Burgers, Fast Food, Sandwiches, Food, Ice Crea...",1,1.0,1,1,1,1,1,1,1,1,1,1,1,1,2.0,1
8,"Pubs, Restaurants, Italian, Bars, American (Tr...",1,0.0,1,1,1,1,1,1,1,1,1,1,1,1,3.0,1
9,"Ice Cream & Frozen Yogurt, Fast Food, Burgers,...",1,1.0,1,1,1,1,0,1,1,1,1,1,1,1,1.5,1
11,"Vietnamese, Food, Restaurants, Food Trucks",1,1.0,1,1,1,1,1,1,1,1,1,1,1,1,4.0,1


# # Data Preprocessing - Categories Column

In [ ]:
# Extract categories column into separate DataFrame
categories_df = business_normalized[['categories']]

# Convert categories column to lowercase and remove leading/trailing white spaces
categories_df['categories'] = categories_df['categories'].str.lower().str.strip()

# Split comma-separated categories into separate columns
categories_df = categories_df['categories'].str.split(',', expand=True)

# Stack the newly created columns
categories_df = categories_df.stack()

# Create dummy variables for each unique category
categories_df = pd.get_dummies(categories_df, prefix='', prefix_sep='')

# Group dummy variables by index and sum them
categories_df = categories_df.groupby(level=0).sum()

# Rename the columns with their original names
categories_df.columns = categories_df.columns.str.replace(' ', 'ct_')

# Group dummy variables by index and sum them
categories_df = categories_df.groupby(level=0).sum()

# Normalize the resulting DataFrame
categories_df = categories_df.div(categories_df.sum(axis=1), axis=0)

#Data cleaning
categories_df = categories_df.applymap(lambda x: 1 if x > 0.0 else 0)
categories_df.drop(columns=['restaurants','ct_restaurants'], inplace=True)

# Merge the new dataframe with the original dataframe
business_normalized = business_normalized.join(categories_df)

#drop categories column
business_normalized.drop(columns=['categories'], inplace=True)

business_normalized.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 86405 entries, 3 to 52267
Columns: 1140 entries, BusinessParking to yoga
dtypes: float64(1126), int64(14)
memory usage: 754.2 MB


In [ ]:
print(business_normalized.columns.tolist())

['BusinessParking', 'is_open', 'OutdoorSeating', 'RestaurantsPriceRange2', 'RestaurantsTakeOut', 'WiFi', 'BusinessAcceptsCreditCards', 'RestaurantsAttire', 'HasTV', 'Ambience', 'GoodForKids', 'RestaurantsGoodForGroups', 'RestaurantsReservations', 'RestaurantsDelivery', 'stars', 'Alcohol', 'ct_acaict_bowls', 'ct_accessories', 'ct_accountants', 'ct_activect_life', 'ct_acupuncture', 'ct_adult', 'ct_adultct_entertainment', 'ct_advertising', 'ct_afghan', 'ct_african', 'ct_airct_ductct_cleaning', 'ct_airlines', 'ct_airportct_lounges', 'ct_airportct_shuttles', 'ct_airportct_terminals', 'ct_airports', 'ct_amateurct_sportsct_teams', 'ct_americanct_(new)', 'ct_americanct_(traditional)', 'ct_amusementct_parks', 'ct_animalct_shelters', 'ct_antiques', 'ct_apartments', 'ct_appliances', 'ct_appliancesct_&ct_repair', 'ct_aquariums', 'ct_arabic', 'ct_arcades', 'ct_argentine', 'ct_armenian', 'ct_artct_classes', 'ct_artct_galleries', 'ct_artct_museums', 'ct_artct_tours', 'ct_artsct_&ct_crafts', 'ct_artsc

In [ ]:
business_normalized.head()

,BusinessParking,is_open,OutdoorSeating,RestaurantsPriceRange2,RestaurantsTakeOut,WiFi,BusinessAcceptsCreditCards,RestaurantsAttire,HasTV,Ambience,...,wholesalect_stores,wholesalers,wigs,winect_bars,winect_tastingct_room,winect_tours,wineries,women'sct_clothing,wraps,yoga
3,1,1.0,1,1,1,1,1,1,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,1,1.0,1,1,1,1,1,1,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,1,0.0,1,1,1,1,1,1,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,1,1.0,1,1,1,1,0,1,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11,1,1.0,1,1,1,1,1,1,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Feature Selection - Categories Column


In [ ]:
from sklearn.feature_selection import SelectKBest, f_classif

#NOTE: Add to sample it because of memory restrictions
business_normalized.fillna(0, inplace=True)
business_feature_selection = business_normalized.sample(n=70000, random_state=42)

# Separate the features and the target variable
X = business_feature_selection.drop(['is_open'], axis=1)
y = business_feature_selection['is_open']

# Select the top 10 features using f_classif score
selector = SelectKBest(score_func=f_classif, k=30)
X_new = selector.fit_transform(X, y)

# Get the selected feature names
selected_features = X.columns[selector.get_support()]

# Print the selected feature names
print('Selected features:', set(selected_features))

Selected features: {'BusinessParking', 'ct_coffeect_&ct_tea', 'ct_nightlife', 'RestaurantsTakeOut', 'BusinessAcceptsCreditCards', 'ct_fastct_food', 'ct_burgers', 'Ambience', 'ct_salad', 'ct_eventct_planningct_&ct_services', 'ct_bars', 'stars', 'ct_americanct_(traditional)', 'ct_food', 'fastct_food', 'RestaurantsGoodForGroups', 'RestaurantsReservations', 'RestaurantsDelivery', 'ct_chickenct_wings', 'ct_mexican', 'ct_breakfastct_&ct_brunch', 'ct_pizza', 'RestaurantsPriceRange2', 'WiFi', 'RestaurantsAttire', 'ct_sandwiches', 'GoodForKids', 'HasTV', 'OutdoorSeating', 'Alcohol'}


In [ ]:
# Get a boolean mask indicating which columns were selected
selected_mask = selector.get_support()

# Select the desired columns using the mask
X_selected_columns = X.columns[selected_mask]

# Drop all columns not selected
business_normalized = pd.concat([business_normalized['is_open'], business_normalized[X_selected_columns]], axis=1)
business_normalized.info()



<class 'pandas.core.frame.DataFrame'>
Int64Index: 86405 entries, 3 to 52267
Data columns (total 31 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   is_open                             86405 non-null  float64
 1   BusinessParking                     86405 non-null  int64  
 2   OutdoorSeating                      86405 non-null  int64  
 3   RestaurantsPriceRange2              86405 non-null  int64  
 4   RestaurantsTakeOut                  86405 non-null  int64  
 5   WiFi                                86405 non-null  int64  
 6   BusinessAcceptsCreditCards          86405 non-null  int64  
 7   RestaurantsAttire                   86405 non-null  int64  
 8   HasTV                               86405 non-null  int64  
 9   Ambience                            86405 non-null  int64  
 10  GoodForKids                         86405 non-null  int64  
 11  RestaurantsGoodForGroups            86405

In [ ]:
business_normalized.head()

,is_open,BusinessParking,OutdoorSeating,RestaurantsPriceRange2,RestaurantsTakeOut,WiFi,BusinessAcceptsCreditCards,RestaurantsAttire,HasTV,Ambience,...,ct_coffeect_&ct_tea,ct_eventct_planningct_&ct_services,ct_fastct_food,ct_food,ct_mexican,ct_nightlife,ct_pizza,ct_salad,ct_sandwiches,fastct_food
3,1.0,1,1,1,1,1,1,1,1,0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
5,1.0,1,1,1,1,1,1,1,1,1,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
8,0.0,1,1,1,1,1,1,1,1,1,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
9,1.0,1,1,1,1,1,0,1,1,1,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
11,1.0,1,1,1,1,1,1,1,1,1,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# Basic Data Cleaning
business.columns = business.columns.str.lower().str.replace(' ', '_') # A
 
string_columns = list(business.dtypes[business.dtypes == 'object'].index) # B
 
for col in string_columns:
    business[col] = business[col].astype(str).str.lower().str.replace(' ', '_')

In [ ]:
business_normalized.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 86405 entries, 3 to 52267
Data columns (total 31 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   is_open                             86405 non-null  float64
 1   BusinessParking                     86405 non-null  int64  
 2   OutdoorSeating                      86405 non-null  int64  
 3   RestaurantsPriceRange2              86405 non-null  int64  
 4   RestaurantsTakeOut                  86405 non-null  int64  
 5   WiFi                                86405 non-null  int64  
 6   BusinessAcceptsCreditCards          86405 non-null  int64  
 7   RestaurantsAttire                   86405 non-null  int64  
 8   HasTV                               86405 non-null  int64  
 9   Ambience                            86405 non-null  int64  
 10  GoodForKids                         86405 non-null  int64  
 11  RestaurantsGoodForGroups            86405

# Machine Learning

# Hyperparameter tuning